In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder, RobustScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, adjusted_rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
import hdbscan
import warnings
warnings.filterwarnings('ignore')

class DemographicClusterer:
    def __init__(self, data_path=None, df=None):
        if df is not None:
            self.df = df.copy()
        else:
            self.df = pd.read_csv(data_path)
        
        self.processed_data = None
        self.cluster_results = {}
    def create_composite_features(self):
        """
        Create composite features for better clustering
        """
        print("Creating composite features...")
        
        # Professional categorization
        profession_mapping = {
            'Doctor': 'High-Skill', 'Engineer': 'High-Skill', 'Executive': 'High-Skill', 
            'Lawyer': 'High-Skill',
            'Artist': 'Creative', 'Entertainment': 'Creative',
            'Healthcare': 'Service/Care', 'Homemaker': 'Service/Care',
            'Marketing': 'Business-Oriented'
        }
        
        if 'profession' in self.df.columns:
            self.df['profession_category'] = self.df['profession'].map(profession_mapping)
        
        # Life Stage Index
        if all(col in self.df.columns for col in ['age', 'work_experience']):
            def determine_life_stage(row):
                age = row.get('age', 0)
                exp = row.get('work_experience', 0)
                marital = row.get('marital_status', 'Unknown')
                
                if age < 25:
                    return 'Career Starter'
                elif 22 <= age <= 35 and marital in ['Single', 'Unmarried'] and exp <= 3:
                    return 'Young Professional'
                elif 30 <= age <= 50 and marital == 'Married' and exp > 3:
                    return 'Established Adult'
                elif age >= 60:
                    return 'Senior Segment'
                else:
                    return 'Transitional'
            
            self.df['life_stage'] = self.df.apply(determine_life_stage, axis=1)
        
        # Career Development Score (0-100 scale)
        if all(col in self.df.columns for col in ['age', 'work_experience']):
            education_weights = {'High School': 1, 'Bachelor': 2, 'Master': 3, 'PhD': 4}
            
            age_score = np.clip((self.df.get('age', 0) - 18) / 47 * 30, 0, 30)  # Max 30 points
            exp_score = np.clip(self.df.get('work_experience', 0) / 20 * 40, 0, 40)  # Max 40 points
            
            if 'education' in self.df.columns:
                edu_score = self.df['education'].map(education_weights).fillna(1) / 4 * 30  # Max 30 points
            else:
                edu_score = 15  # Default middle score
            
            self.df['career_development_score'] = age_score + exp_score + edu_score
        
        # Economic Capacity Index (0-100 scale)
        if 'profession_category' in self.df.columns:
            profession_weights = {'High-Skill': 4, 'Business-Oriented': 3, 'Creative': 2, 'Service/Care': 1}
            prof_score = self.df['profession_category'].map(profession_weights).fillna(2) / 4 * 40
            
            if 'education' in self.df.columns:
                edu_score = self.df['education'].map(education_weights).fillna(2) / 4 * 30
            else:
                edu_score = 15
            
            spending_weights = {'Low': 1, 'Average': 2, 'High': 3}
            if 'spending_score' in self.df.columns:
                spend_score = self.df['spending_score'].map(spending_weights).fillna(2) / 3 * 30
            elif 'annual_spending' in self.df.columns:
                # Normalize annual spending to 0-30 scale
                spend_score = (self.df['annual_spending'] - self.df['annual_spending'].min()) / \
                             (self.df['annual_spending'].max() - self.df['annual_spending'].min()) * 30
            else:
                spend_score = 15
            
            self.df['economic_capacity_index'] = prof_score + edu_score + spend_score
        
        # Family Responsibility Factor
        if 'age' in self.df.columns:
            age_weight = np.clip(self.df['age'] / 70, 0.5, 1.5)  # Age factor
            
            marital_weight = 1.0
            if 'marital_status' in self.df.columns:
                marital_weights = {'Married': 1.5, 'Single': 0.8, 'Divorced': 1.2, 'Widowed': 1.0}
                marital_weight = self.df['marital_status'].map(marital_weights).fillna(1.0)
            
            family_size = 1
            if 'family_size' in self.df.columns:
                family_size = self.df['family_size'].fillna(1)
            
            self.df['family_responsibility_factor'] = age_weight * marital_weight * family_size
        
        # Profession-Education Alignment
        if all(col in self.df.columns for col in ['profession_category', 'education']):
            def check_alignment(row):
                prof_cat = row.get('profession_category', '')
                education = row.get('education', '')
                
                if prof_cat == 'High-Skill' and education in ['Bachelor', 'Master', 'PhD']:
                    return 1
                elif prof_cat in ['Creative', 'Business-Oriented'] and education in ['Bachelor', 'Master']:
                    return 1
                elif prof_cat == 'Service/Care':
                    return 1  # Service roles have varied education requirements
                else:
                    return 0
            
            self.df['profession_education_match'] = self.df.apply(check_alignment, axis=1)
        
        print(f"Created composite features. New shape: {self.df.shape}")
        return self.df
        """
        Handle mixed data types for clustering
        """
        print("Starting data preprocessing...")
        
        if categorical_cols is None:
            categorical_cols = self.df.select_dtypes(include=['object', 'category']).columns.tolist()
        if numerical_cols is None:
            numerical_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        
        self.preprocessing_info['categorical_cols'] = categorical_cols
        self.preprocessing_info['numerical_cols'] = numerical_cols
        
        # Create preprocessing pipeline
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numerical_cols),
                ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
            ])
        
        self.processed_data = preprocessor.fit_transform(self.df)
        self.preprocessor = preprocessor
        
        # Get feature names after preprocessing
        num_features = numerical_cols
        cat_features = []
        if len(categorical_cols) > 0:
            cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
        
        self.feature_names = num_features + list(cat_features)
        
        print(f"Data shape after preprocessing: {self.processed_data.shape}")
        print(f"Features: {len(self.feature_names)} total")
        return self.processed_data
    
    def find_optimal_clusters(self, max_clusters=10):
        """
        Use elbow method and silhouette analysis to find optimal number of clusters
        """
        inertias = []
        silhouette_scores = []
        k_range = range(2, max_clusters + 1)
        
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(self.processed_data)
            
            inertias.append(kmeans.inertia_)
            sil_score = silhouette_score(self.processed_data, labels)
            silhouette_scores.append(sil_score)
        
        # Plot elbow curve and silhouette scores
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        ax1.plot(k_range, inertias, 'bo-')
        ax1.set_xlabel('Number of Clusters')
        ax1.set_ylabel('Inertia')
        ax1.set_title('Elbow Method')
        ax1.grid(True, alpha=0.3)
        
        ax2.plot(k_range, silhouette_scores, 'ro-')
        ax2.set_xlabel('Number of Clusters')
        ax2.set_ylabel('Silhouette Score')
        ax2.set_title('Silhouette Analysis')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Suggest optimal k
        optimal_k = k_range[np.argmax(silhouette_scores)]
        
    def gap_statistic(self, max_clusters=10, n_refs=20):
        """
        Calculate Gap Statistic for optimal cluster number determination
        """
        print("Calculating Gap Statistic...")
        
        gaps = []
        results_df = []
        
        for k in range(1, max_clusters + 1):
            # Cluster the data
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(self.processed_data)
            
            # Calculate within-cluster sum of squares
            wk = sum([np.sum((self.processed_data[labels == i] - kmeans.cluster_centers_[i])**2) 
                     for i in range(k)])
            
            # Generate reference datasets and calculate their within-cluster sum of squares
            ref_wks = []
            for _ in range(n_refs):
                # Generate random reference data with same shape
                ref_data = np.random.uniform(
                    low=self.processed_data.min(axis=0),
                    high=self.processed_data.max(axis=0),
                    size=self.processed_data.shape
                )
                
                ref_kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
                ref_labels = ref_kmeans.fit_predict(ref_data)
                
                ref_wk = sum([np.sum((ref_data[ref_labels == i] - ref_kmeans.cluster_centers_[i])**2) 
                             for i in range(k)])
                ref_wks.append(ref_wk)
            
            # Calculate gap
            gap = np.log(np.mean(ref_wks)) - np.log(wk)
            gaps.append(gap)
            
            results_df.append({
                'k': k,
                'gap': gap,
                'log_wk': np.log(wk),
                'ref_log_wk': np.log(np.mean(ref_wks))
            })
        
        # Find optimal k (first k where gap(k) >= gap(k+1) - s(k+1))
        optimal_k = 1
        for i in range(len(gaps) - 1):
            if gaps[i] >= gaps[i + 1]:  # Simplified rule
                optimal_k = i + 1
                break
        
        # Plot gap statistic
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, max_clusters + 1), gaps, 'bo-')
        plt.axvline(x=optimal_k, color='r', linestyle='--', alpha=0.7, label=f'Optimal k={optimal_k}')
        plt.xlabel('Number of Clusters (k)')
        plt.ylabel('Gap Statistic')
        plt.title('Gap Statistic for Optimal Number of Clusters')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print(f"Gap Statistic suggests optimal k: {optimal_k}")
        return optimal_k, gaps
    
    def apply_clustering_methods(self, n_clusters=4):
        """
        Apply enhanced clustering algorithms with HDBSCAN as primary method
        """
        print("Applying enhanced clustering methods...")
        
        methods = {}
        
        # 1. HDBSCAN (Primary) - Density-based clustering
        print("Applying HDBSCAN...")
        hdbscan_clusterer = hdbscan.HDBSCAN(
            min_cluster_size=max(5, len(self.processed_data) // 50),  # Adaptive min size
            min_samples=3,
            cluster_selection_epsilon=0.5,
            cluster_selection_method='eom'
        )
        hdbscan_labels = hdbscan_clusterer.fit_predict(self.processed_data)
        
        methods['HDBSCAN'] = {
            'model': hdbscan_clusterer,
            'labels': hdbscan_labels,
            'n_clusters': len(set(hdbscan_labels)) - (1 if -1 in hdbscan_labels else 0),
            'noise_points': np.sum(hdbscan_labels == -1),
            'cluster_probabilities': hdbscan_clusterer.probabilities_ if hasattr(hdbscan_clusterer, 'probabilities_') else None
        }
        
        # 2. Gaussian Mixture Models (Secondary) - Probabilistic clustering
        print("Applying Gaussian Mixture...")
        gmm = GaussianMixture(n_components=n_clusters, random_state=42, covariance_type='full')
        gmm.fit(self.processed_data)
        gmm_labels = gmm.predict(self.processed_data)
        gmm_probs = gmm.predict_proba(self.processed_data)
        
        methods['Gaussian Mixture'] = {
            'model': gmm,
            'labels': gmm_labels,
            'n_clusters': n_clusters,
            'probabilities': gmm_probs,
            'aic': gmm.aic(self.processed_data),
            'bic': gmm.bic(self.processed_data)
        }
        
        # 3. K-Means (Validation) - Traditional centroid-based
        print("Applying K-Means...")
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=20)
        kmeans_labels = kmeans.fit_predict(self.processed_data)
        
        methods['K-Means'] = {
            'model': kmeans,
            'labels': kmeans_labels,
            'n_clusters': n_clusters,
            'inertia': kmeans.inertia_,
            'cluster_centers': kmeans.cluster_centers_
        }
        
        # 4. Hierarchical (Exploratory) - For understanding relationships
        print("Applying Hierarchical Clustering...")
        hierarchical = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
        hierarchical_labels = hierarchical.fit_predict(self.processed_data)
        
        methods['Hierarchical'] = {
            'model': hierarchical,
            'labels': hierarchical_labels,
            'n_clusters': n_clusters
        }
        
        # Calculate comprehensive metrics for each method
        for name, result in methods.items():
            labels = result['labels']
            
            # Skip metrics calculation if only one cluster or all noise
            if len(np.unique(labels)) <= 1 or (len(np.unique(labels)) == 2 and -1 in labels):
                result.update({
                    'silhouette_score': -1,
                    'calinski_harabasz_score': 0,
                    'davies_bouldin_score': float('inf'),
                    'stability_score': 0
                })
                continue
            
            # Standard metrics
            if len(np.unique(labels)) > 1:
                # Handle noise points for HDBSCAN
                if -1 in labels:
                    mask = labels != -1
                    if np.sum(mask) > 1 and len(np.unique(labels[mask])) > 1:
                        sil_score = silhouette_score(self.processed_data[mask], labels[mask])
                        ch_score = calinski_harabasz_score(self.processed_data[mask], labels[mask])
                        db_score = davies_bouldin_score(self.processed_data[mask], labels[mask])
                    else:
                        sil_score = ch_score = db_score = -1
                else:
                    sil_score = silhouette_score(self.processed_data, labels)
                    ch_score = calinski_harabasz_score(self.processed_data, labels)
                    db_score = davies_bouldin_score(self.processed_data, labels)
            else:
                sil_score = ch_score = db_score = -1
            
            # Stability analysis using bootstrap sampling
            stability_score = self.calculate_stability(result['model'], name)
            
            result.update({
                'silhouette_score': sil_score,
                'calinski_harabasz_score': ch_score,
                'davies_bouldin_score': db_score,
                'stability_score': stability_score
            })
        
        self.cluster_results = methods
        return methods
    
    def calculate_stability(self, model, method_name, n_bootstrap=10):
        """
        Calculate clustering stability using bootstrap sampling
        """
        try:
            n_samples = len(self.processed_data)
            stability_scores = []
            
            for _ in range(n_bootstrap):
                # Bootstrap sample
                indices = np.random.choice(n_samples, size=int(0.8 * n_samples), replace=True)
                bootstrap_data = self.processed_data[indices]
                
                # Apply clustering to bootstrap sample
                if method_name == 'HDBSCAN':
                    labels = model.fit_predict(bootstrap_data)
                elif method_name == 'Gaussian Mixture':
                    temp_model = GaussianMixture(n_components=model.n_components, random_state=42)
                    temp_model.fit(bootstrap_data)
                    labels = temp_model.predict(bootstrap_data)
                elif method_name == 'K-Means':
                    temp_model = KMeans(n_clusters=model.n_clusters, random_state=42, n_init=10)
                    labels = temp_model.fit_predict(bootstrap_data)
                elif method_name == 'Hierarchical':
                    temp_model = AgglomerativeClustering(n_clusters=model.n_clusters)
                    labels = temp_model.fit_predict(bootstrap_data)
                
                # Calculate silhouette score if possible
                if len(np.unique(labels)) > 1:
                    if -1 in labels:  # Handle noise points
                        mask = labels != -1
                        if np.sum(mask) > 1 and len(np.unique(labels[mask])) > 1:
                            score = silhouette_score(bootstrap_data[mask], labels[mask])
                        else:
                            score = -1
                    else:
                        score = silhouette_score(bootstrap_data, labels)
                    stability_scores.append(score)
            
            return np.mean(stability_scores) if stability_scores else 0
        except:
            return 0
    
    def evaluate_clusters(self):
        """
        Enhanced cluster evaluation with business validation
        """
        print("\n" + "="*70)
        print("COMPREHENSIVE CLUSTERING EVALUATION")
        print("="*70)
        
        evaluation_results = []
        
        for method, result in self.cluster_results.items():
            labels = result['labels']
            n_clusters = result['n_clusters']
            
            # Basic metrics
            sil_score = result.get('silhouette_score', -1)
            ch_score = result.get('calinski_harabasz_score', 0)
            db_score = result.get('davies_bouldin_score', float('inf'))
            stability = result.get('stability_score', 0)
            
            # Business validation metrics
            cluster_sizes = np.bincount(labels[labels >= 0])  # Exclude noise points
            min_cluster_size_pct = (np.min(cluster_sizes) / len(labels)) * 100 if len(cluster_sizes) > 0 else 0
            
            # Actionability score (based on feature separation)
            actionability_score = self.calculate_actionability_score(labels)
            
            # Overall score (weighted combination)
            if sil_score > 0:
                overall_score = (
                    0.3 * sil_score +
                    0.2 * min(ch_score / 1000, 1) +  # Normalized CH score
                    0.2 * max(0, 1 - db_score / 5) +  # Inverted DB score
                    0.15 * stability +
                    0.15 * actionability_score
                )
            else:
                overall_score = 0
            
            evaluation_results.append({
                'Method': method,
                'Clusters': n_clusters,
                'Silhouette': f"{sil_score:.3f}" if sil_score > 0 else "N/A",
                'CH Score': f"{ch_score:.1f}" if ch_score > 0 else "N/A",
                'DB Score': f"{db_score:.3f}" if db_score != float('inf') else "N/A",
                'Stability': f"{stability:.3f}",
                'Min Size %': f"{min_cluster_size_pct:.1f}%",
                'Actionability': f"{actionability_score:.3f}",
                'Overall Score': f"{overall_score:.3f}",
                'Noise Points': result.get('noise_points', 0)
            })
        
        eval_df = pd.DataFrame(evaluation_results)
        print(eval_df.to_string(index=False))
        
        # Business validation checks
        print(f"\n{'='*50}")
        print("BUSINESS VALIDATION CHECKS")
        print(f"{'='*50}")
        
        for method, result in self.cluster_results.items():
            labels = result['labels']
            cluster_sizes = np.bincount(labels[labels >= 0])
            
            print(f"\n{method}:")
            print(f"  ✓ Number of clusters: {result['n_clusters']}")
            
            # Size validation
            min_size_pct = (np.min(cluster_sizes) / len(labels)) * 100 if len(cluster_sizes) > 0 else 0
            if min_size_pct >= 5:
                print(f"  ✓ All clusters >5% of population (min: {min_size_pct:.1f}%)")
            else:
                print(f"  ✗ Some clusters <5% of population (min: {min_size_pct:.1f}%)")
            
            # Stability validation
            stability = result.get('stability_score', 0)
            if stability >= 0.3:
                print(f"  ✓ Good stability (score: {stability:.3f})")
            else:
                print(f"  ✗ Low stability (score: {stability:.3f})")
        
        # Recommend best method
        if evaluation_results:
            best_method_row = max(evaluation_results, key=lambda x: float(x['Overall Score']))
            best_method = best_method_row['Method']
            print(f"\n🏆 RECOMMENDED METHOD: {best_method}")
            print(f"   Overall Score: {best_method_row['Overall Score']}")
            print(f"   Clusters: {best_method_row['Clusters']}")
            print(f"   Silhouette: {best_method_row['Silhouette']}")
            
        return eval_df
    
    def calculate_actionability_score(self, labels):
        """
        Calculate how actionable/interpretable the clusters are
        """
        try:
            # Use original features for interpretability
            data_to_use = self.processed_data_original if hasattr(self, 'processed_data_original') else self.processed_data
            
            if len(np.unique(labels)) <= 1:
                return 0
            
            # Train a decision tree to see how well features separate clusters
            mask = labels >= 0  # Exclude noise points
            if np.sum(mask) == 0:
                return 0
                
            tree = DecisionTreeClassifier(max_depth=5, random_state=42)
            tree.fit(data_to_use[mask], labels[mask])
            
            # Feature importance as proxy for actionability
            feature_importance = np.mean(tree.feature_importances_)
            
            # Accuracy as separation quality
            accuracy = tree.score(data_to_use[mask], labels[mask])
            
            return (feature_importance + accuracy) / 2
        except:
            return 0
    
    def create_advanced_tsne_visualization(self, method='HDBSCAN', perplexity_values=[30, 50], plot_3d=False):
        """
        Create advanced t-SNE visualization with multiple perplexity values
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        n_perplexity = len(perplexity_values)
        
        if plot_3d:
            from mpl_toolkits.mplot3d import Axes3D
            
            fig = plt.figure(figsize=(15, 5 * n_perplexity))
            
            for i, perplexity in enumerate(perplexity_values):
                print(f"Creating 3D t-SNE with perplexity={perplexity}...")
                
                tsne = TSNE(n_components=3, random_state=42, perplexity=perplexity)
                tsne_data = tsne.fit_transform(self.processed_data)
                
                ax = fig.add_subplot(n_perplexity, 1, i+1, projection='3d')
                scatter = ax.scatter(tsne_data[:, 0], tsne_data[:, 1], tsne_data[:, 2],
                                  c=labels, cmap='viridis', alpha=0.7, s=50)
                ax.set_title(f'3D t-SNE - {method} (perplexity={perplexity})')
                ax.set_xlabel('t-SNE 1')
                ax.set_ylabel('t-SNE 2')
                ax.set_zlabel('t-SNE 3')
                plt.colorbar(scatter, ax=ax, shrink=0.5)
        else:
            fig, axes = plt.subplots(1, n_perplexity, figsize=(7 * n_perplexity, 6))
            if n_perplexity == 1:
                axes = [axes]
            
            for i, perplexity in enumerate(perplexity_values):
                print(f"Creating 2D t-SNE with perplexity={perplexity}...")
                
                tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
                tsne_data = tsne.fit_transform(self.processed_data)
                
                ax = axes[i]
                scatter = ax.scatter(tsne_data[:, 0], tsne_data[:, 1], 
                                   c=labels, cmap='viridis', alpha=0.7, s=50)
                ax.set_title(f't-SNE - {method} (perplexity={perplexity})')
                ax.set_xlabel('t-SNE Component 1')
                ax.set_ylabel('t-SNE Component 2')
                
                # Add cluster centers
                unique_labels = np.unique(labels[labels >= 0])  # Exclude noise
                for label in unique_labels:
                    mask = labels == label
                    if np.sum(mask) > 0:
                        center_x = np.mean(tsne_data[mask, 0])
                        center_y = np.mean(tsne_data[mask, 1])
                        ax.annotate(f'C{label}', (center_x, center_y), 
                                  fontsize=12, fontweight='bold',
                                  bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
                
                plt.colorbar(scatter, ax=ax)
        
        plt.tight_layout()
        plt.show()
    
    def create_feature_contribution_plot(self, method='HDBSCAN'):
        """
        Create feature contribution plots showing which features drive clustering
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        
        # Use original features for interpretability
        if hasattr(self, 'processed_data_original'):
            data_to_use = self.processed_data_original
            feature_names = self.feature_names
        else:
            data_to_use = self.processed_data
            feature_names = getattr(self, 'pca_feature_names', [f'Feature_{i}' for i in range(data_to_use.shape[1])])
        
        # Train a decision tree to understand feature importance
        mask = labels >= 0  # Exclude noise points
        if np.sum(mask) == 0:
            print("No valid clusters to analyze")
            return
        
        tree = DecisionTreeClassifier(max_depth=4, random_state=42)
        tree.fit(data_to_use[mask], labels[mask])
        
        # Plot feature importance
        importances = tree.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        plt.figure(figsize=(12, 8))
        
        # Feature importance bar plot
        plt.subplot(2, 2, 1)
        plt.bar(range(min(10, len(importances))), importances[indices[:10]])
        plt.title('Top 10 Feature Importances for Clustering')
        plt.xlabel('Features')
        plt.ylabel('Importance')
        feature_labels = [feature_names[i] if i < len(feature_names) else f'Feature_{i}' for i in indices[:10]]
        plt.xticks(range(min(10, len(importances))), feature_labels, rotation=45, ha='right')
        
        # Decision tree visualization (simplified)
        plt.subplot(2, 2, 2)
        plot_tree(tree, max_depth=2, feature_names=feature_names[:len(feature_names)], 
                 class_names=[f'Cluster {i}' for i in np.unique(labels[mask])],
                 filled=True, rounded=True, fontsize=8)
        plt.title('Decision Tree (Depth 2)')
        
        # Cluster size distribution
        plt.subplot(2, 2, 3)
        cluster_counts = np.bincount(labels[labels >= 0])
        cluster_ids = np.arange(len(cluster_counts))
        plt.bar(cluster_ids, cluster_counts)
        plt.title('Cluster Size Distribution')
        plt.xlabel('Cluster ID')
        plt.ylabel('Number of Points')
        
        # Feature correlation with clusters
        plt.subplot(2, 2, 4)
        if len(feature_names) > 0:
            # Calculate correlation between first few features and cluster assignments
            correlations = []
            for i in range(min(5, data_to_use.shape[1])):
                corr = np.corrcoef(data_to_use[mask, i], labels[mask])[0, 1]
                correlations.append(abs(corr))
            
            plt.bar(range(len(correlations)), correlations)
            plt.title('Feature-Cluster Correlations')
            plt.xlabel('Features')
            plt.ylabel('|Correlation| with Clusters')
            plt.xticks(range(len(correlations)), 
                      [feature_names[i][:10] if i < len(feature_names) else f'F{i}' for i in range(len(correlations))], 
                      rotation=45, ha='right')
        
        plt.tight_layout()
        plt.show()
    
    def create_business_dashboard(self, method='HDBSCAN'):
        """
        Create a business-focused dashboard showing cluster characteristics
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        df_with_clusters = self.df.copy()
        df_with_clusters['Cluster'] = labels
        
        # Filter out noise points
        df_filtered = df_with_clusters[df_with_clusters['Cluster'] >= 0]
        
        if len(df_filtered) == 0:
            print("No valid clusters to display")
            return
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        # 1. Cluster size and stability
        ax = axes[0]
        cluster_counts = df_filtered['Cluster'].value_counts().sort_index()
        bars = ax.bar(cluster_counts.index, cluster_counts.values)
        ax.set_title('Cluster Sizes', fontsize=14, fontweight='bold')
        ax.set_xlabel('Cluster ID')
        ax.set_ylabel('Number of Customers')
        
        # Add percentage labels
        total = len(df_filtered)
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + total*0.01,
                   f'{height/total*100:.1f}%', ha='center', va='bottom')
        
        # 2. Age distribution by cluster
        if 'age' in df_filtered.columns:
            ax = axes[1]
            df_filtered.boxplot(column='age', by='Cluster', ax=ax)
            ax.set_title('Age Distribution by Cluster')
            ax.set_xlabel('Cluster')
            ax.set_ylabel('Age')
        
        # 3. Spending patterns
        spending_col = None
        if 'annual_spending' in df_filtered.columns:
            spending_col = 'annual_spending'
        elif 'spending_score' in df_filtered.columns:
            spending_col = 'spending_score'
        
        if spending_col:
            ax = axes[2]
            if spending_col == 'spending_score':
                # Categorical spending
                spending_crosstab = pd.crosstab(df_filtered['Cluster'], df_filtered[spending_col], normalize='index')
                spending_crosstab.plot(kind='bar', stacked=True, ax=ax)
                ax.set_title('Spending Score Distribution by Cluster')
                ax.set_ylabel('Proportion')
            else:
                # Numerical spending
                df_filtered.boxplot(column=spending_col, by='Cluster', ax=ax)
                ax.set_title('Spending Distribution by Cluster')
                ax.set_ylabel('Annual Spending')
            ax.set_xlabel('Cluster')
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
        
        # 4. Education levels
        if 'education' in df_filtered.columns:
            ax = axes[3]
            edu_crosstab = pd.crosstab(df_filtered['Cluster'], df_filtered['education'], normalize='index')
            edu_crosstab.plot(kind='bar', stacked=True, ax=ax)
            ax.set_title('Education Distribution by Cluster')
            ax.set_xlabel('Cluster')
            ax.set_ylabel('Proportion')
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
        
        # 5. Life stage distribution
        if 'life_stage' in df_filtered.columns:
            ax = axes[4]
            life_crosstab = pd.crosstab(df_filtered['Cluster'], df_filtered['life_stage'], normalize='index')
            life_crosstab.plot(kind='bar', stacked=True, ax=ax)
            ax.set_title('Life Stage Distribution by Cluster')
            ax.set_xlabel('Cluster')
            ax.set_ylabel('Proportion')
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=0)
        
        # 6. Composite scores
        if 'economic_capacity_index' in df_filtered.columns:
            ax = axes[5]
            df_filtered.boxplot(column='economic_capacity_index', by='Cluster', ax=ax)
            ax.set_title('Economic Capacity by Cluster')
            ax.set_xlabel('Cluster')
            ax.set_ylabel('Economic Capacity Index')
        
        plt.tight_layout()
        plt.show()
        
        # Print cluster summaries
        print(f"\n{'='*60}")
        print(f"BUSINESS CLUSTER SUMMARIES - {method}")
        print(f"{'='*60}")
        
        for cluster_id in sorted(df_filtered['Cluster'].unique()):
            cluster_data = df_filtered[df_filtered['Cluster'] == cluster_id]
            pct = len(cluster_data) / len(df_filtered) * 100
            
            print(f"\n🎯 CLUSTER {cluster_id} ({len(cluster_data)} customers, {pct:.1f}%)")
            print("-" * 50)
            
            # Demographics
            if 'age' in cluster_data.columns:
                print(f"Age: {cluster_data['age'].mean():.1f} ± {cluster_data['age'].std():.1f} years")
            
            # Life stage
            if 'life_stage' in cluster_data.columns:
                top_life_stage = cluster_data['life_stage'].mode().iloc[0] if not cluster_data['life_stage'].mode().empty else 'Unknown'
                print(f"Primary Life Stage: {top_life_stage}")
            
            # Economic indicators
            if 'economic_capacity_index' in cluster_data.columns:
                print(f"Economic Capacity: {cluster_data['economic_capacity_index'].mean():.1f}/100")
            
            # Spending
            if spending_col:
                if spending_col == 'spending_score':
                    top_spending = cluster_data[spending_col].mode().iloc[0] if not cluster_data[spending_col].mode().empty else 'Unknown'
                    print(f"Spending Level: {top_spending}")
                else:
                    print(f"Average Spending: ${cluster_data[spending_col].mean():.0f}")
            
            # Professional category
            if 'profession_category' in cluster_data.columns:
                top_profession = cluster_data['profession_category'].mode().iloc[0] if not cluster_data['profession_category'].mode().empty else 'Unknown'
                print(f"Primary Profession Type: {top_profession}")
    
    def generate_segment_names(self, method='HDBSCAN'):
        """
        Generate descriptive names for clusters based on their characteristics
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        df_with_clusters = self.df.copy()
        df_with_clusters['Cluster'] = labels
        
        # Filter out noise points
        df_filtered = df_with_clusters[df_with_clusters['Cluster'] >= 0]
        
        segment_names = {}
        
        for cluster_id in sorted(df_filtered['Cluster'].unique()):
            cluster_data = df_filtered[df_filtered['Cluster'] == cluster_id]
            
            # Analyze key characteristics
            descriptors = []
            
            # Age-based descriptor
            if 'age' in cluster_data.columns:
                avg_age = cluster_data['age'].mean()
                if avg_age < 30:
                    descriptors.append('Young')
                elif avg_age > 50:
                    descriptors.append('Senior')
                else:
                    descriptors.append('Mid-Career')
            
            # Profession-based descriptor
            if 'profession_category' in cluster_data.columns:
                top_profession = cluster_data['profession_category'].mode().iloc[0] if not cluster_data['profession_category'].mode().empty else None
                if top_profession:
                    if top_profession == 'High-Skill':
                        descriptors.append('Professionals')
                    elif top_profession == 'Creative':
                        descriptors.append('Creatives')
                    elif top_profession == 'Service/Care':
                        descriptors.append('Service Workers')
                    elif top_profession == 'Business-Oriented':
                        descriptors.append('Business')
            
            # Spending-based descriptor
            spending_col = 'annual_spending' if 'annual_spending' in cluster_data.columns else 'spending_score'
            if spending_col in cluster_data.columns:
                if spending_col == 'spending_score':
                    top_spending = cluster_data[spending_col].mode().iloc[0] if not cluster_data[spending_col].mode().empty else None
                    if top_spending == 'High':
                        descriptors.append('High Spenders')
                    elif top_spending == 'Low':
                        descriptors.append('Budget Conscious')
                else:
                    avg_spending = cluster_data[spending_col].mean()
                    overall_avg = df_filtered[spending_col].mean()
                    if avg_spending > overall_avg * 1.2:
                        descriptors.append('High Spenders')
                    elif avg_spending < overall_avg * 0.8:
                        descriptors.append('Budget Conscious')
            
            # Generate name
            if len(descriptors) >= 2:
                name = ' '.join(descriptors[:2])
            elif len(descriptors) == 1:
                name = descriptors[0]
            else:
                name = f'Segment {cluster_id}'
            
            segment_names[cluster_id] = name
        
        print(f"\n🏷️  SUGGESTED SEGMENT NAMES - {method}")
        print("-" * 50)
        for cluster_id, name in segment_names.items():
            cluster_size = len(df_filtered[df_filtered['Cluster'] == cluster_id])
            pct = cluster_size / len(df_filtered) * 100
            print(f"Cluster {cluster_id}: '{name}' ({cluster_size} customers, {pct:.1f}%)")
        
        return segment_names
        """
        Create t-SNE visualization of clusters
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        print(f"Creating t-SNE visualization for {method}...")
        
        # Apply t-SNE
        tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
        tsne_data = tsne.fit_transform(self.processed_data)
        
        labels = self.cluster_results[method]['labels']
        
        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(tsne_data[:, 0], tsne_data[:, 1], 
                            c=labels, cmap='viridis', alpha=0.7, s=50)
        plt.colorbar(scatter)
        plt.title(f't-SNE Visualization - {method}')
        plt.xlabel('t-SNE Component 1')
        plt.ylabel('t-SNE Component 2')
        
        # Add cluster centers if available
        unique_labels = np.unique(labels)
        for label in unique_labels:
            mask = labels == label
            center_x = np.mean(tsne_data[mask, 0])
            center_y = np.mean(tsne_data[mask, 1])
            plt.annotate(f'C{label}', (center_x, center_y), 
                        fontsize=12, fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        
        plt.tight_layout()
        plt.show()
    
    def create_cluster_profiles(self, method='K-Means'):
        """
        Create detailed cluster profiles showing characteristics
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        df_with_clusters = self.df.copy()
        df_with_clusters['Cluster'] = labels
        
        print(f"\n{'='*50}")
        print(f"CLUSTER PROFILES - {method}")
        print(f"{'='*50}")
        
        # Numerical features analysis
        numerical_cols = self.preprocessing_info['numerical_cols']
        if numerical_cols:
            print("\nNumerical Features by Cluster:")
            numerical_stats = df_with_clusters.groupby('Cluster')[numerical_cols].agg(['mean', 'std'])
            print(numerical_stats.round(2))
        
        # Categorical features analysis
        categorical_cols = self.preprocessing_info['categorical_cols']
        if categorical_cols:
            print("\nCategorical Features Distribution by Cluster:")
            for col in categorical_cols:
                print(f"\n{col.upper()}:")
                crosstab = pd.crosstab(df_with_clusters['Cluster'], 
                                     df_with_clusters[col], normalize='index')
                print((crosstab * 100).round(1).to_string())
        
        return df_with_clusters
    
    def create_radial_charts(self, method='K-Means', sample_features=None):
        """
        Create radial/radar charts for cluster comparison
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        df_with_clusters = self.df.copy()
        df_with_clusters['Cluster'] = labels
        
        # Select features for radial chart
        numerical_cols = self.preprocessing_info['numerical_cols']
        if sample_features is None:
            sample_features = numerical_cols[:6]  # Limit to 6 features for readability
        
        if not sample_features:
            print("No numerical features available for radial chart")
            return
        
        # Calculate means for each cluster
        cluster_means = df_with_clusters.groupby('Cluster')[sample_features].mean()
        
        # Normalize values to 0-1 scale for better visualization
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        normalized_means = pd.DataFrame(
            scaler.fit_transform(cluster_means),
            columns=cluster_means.columns,
            index=cluster_means.index
        )
        
        # Create radial charts with automatic grid layout
        n_clusters = len(normalized_means)
        
        # Calculate optimal grid dimensions
        if n_clusters <= 3:
            n_rows, n_cols = 1, n_clusters
        elif n_clusters <= 6:
            n_rows, n_cols = 2, (n_clusters + 1) // 2
        elif n_clusters <= 9:
            n_rows, n_cols = 3, (n_clusters + 2) // 3
        else:
            n_rows = int(np.ceil(np.sqrt(n_clusters)))
            n_cols = int(np.ceil(n_clusters / n_rows))
        
        # Adjust figure size based on grid
        fig_width = min(5 * n_cols, 20)  # Cap width at 20
        fig_height = min(5 * n_rows, 15)  # Cap height at 15
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), 
                               subplot_kw=dict(projection='polar'))
        
        # Handle different cases for axes array structure
        if n_clusters == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = axes if n_cols > 1 else [axes]
        else:
            axes = axes.flatten()
        
        angles = np.linspace(0, 2*np.pi, len(sample_features), endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle
        
        colors = plt.cm.Set3(np.linspace(0, 1, n_clusters))
        
        for idx, (cluster_id, cluster_data) in enumerate(normalized_means.iterrows()):
            values = cluster_data.tolist()
            values += values[:1]  # Complete the circle
            
            ax = axes[idx]
            ax.plot(angles, values, 'o-', linewidth=2, color=colors[idx])
            ax.fill(angles, values, alpha=0.25, color=colors[idx])
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels(sample_features)
            ax.set_ylim(0, 1)
            ax.set_title(f'Cluster {cluster_id}', size=14, weight='bold', pad=20)
            ax.grid(True)
        
        # Hide unused subplots if any
        total_subplots = n_rows * n_cols
        if n_clusters < total_subplots:
            for i in range(n_clusters, total_subplots):
                axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    def create_comparison_heatmap(self, method='K-Means'):
        """
        Create heatmap showing cluster characteristics
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        df_with_clusters = self.df.copy()
        df_with_clusters['Cluster'] = labels
        
        # Get numerical features means by cluster
        numerical_cols = self.preprocessing_info['numerical_cols']
        if not numerical_cols:
            print("No numerical features for heatmap")
            return
        
        cluster_means = df_with_clusters.groupby('Cluster')[numerical_cols].mean()
        
        # Normalize for better visualization
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        normalized_data = scaler.fit_transform(cluster_means.T).T
        
        plt.figure(figsize=(10, 6))
        sns.heatmap(normalized_data, 
                   xticklabels=numerical_cols,
                   yticklabels=[f'Cluster {i}' for i in cluster_means.index],
                   annot=True, fmt='.2f', cmap='RdYlBu_r', center=0)
        plt.title(f'Cluster Characteristics Heatmap - {method}')
        plt.tight_layout()
        plt.show()

    def create_enhanced_radial_charts(self, method='HDBSCAN', sample_features=None, show_comparison=True):
        """
        Create enhanced radial charts with feature importance weighting and comparison mode
        """
        if method not in self.cluster_results:
            print(f"Method {method} not found in results")
            return
        
        labels = self.cluster_results[method]['labels']
        df_with_clusters = self.df.copy()
        df_with_clusters['Cluster'] = labels
        
        # Filter out noise points
        df_filtered = df_with_clusters[df_with_clusters['Cluster'] >= 0]
        
        # Select features for radial chart - prioritize composite features
        composite_features = ['career_development_score', 'economic_capacity_index', 'family_responsibility_factor']
        numerical_cols = [col for col in self.preprocessing_info.get('numerical_cols', []) if col in df_filtered.columns]
        
        if sample_features is None:
            # Prioritize composite features, then numerical
            available_composite = [f for f in composite_features if f in df_filtered.columns]
            available_numerical = [f for f in numerical_cols if f not in available_composite]
            sample_features = available_composite + available_numerical[:6-len(available_composite)]
            sample_features = sample_features[:6]  # Limit to 6 for readability
        
        if not sample_features:
            print("No suitable features available for radial chart")
            return
        
        # Calculate means for each cluster
        cluster_means = df_filtered.groupby('Cluster')[sample_features].mean()
        
        # Normalize values to 0-1 scale
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        normalized_means = pd.DataFrame(
            scaler.fit_transform(cluster_means),
            columns=cluster_means.columns,
            index=cluster_means.index
        )
        
        if show_comparison:
            # Single plot comparing all clusters
            fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection='polar'))
            
            angles = np.linspace(0, 2*np.pi, len(sample_features), endpoint=False).tolist()
            angles += angles[:1]  # Complete the circle
            
            colors = plt.cm.Set3(np.linspace(0, 1, len(normalized_means)))
            
            for idx, (cluster_id, cluster_data) in enumerate(normalized_means.iterrows()):
                values = cluster_data.tolist()
                values += values[:1]  # Complete the circle
                
                ax.plot(angles, values, 'o-', linewidth=3, color=colors[idx], 
                       label=f'Cluster {cluster_id}', alpha=0.8)
                ax.fill(angles, values, alpha=0.15, color=colors[idx])
            
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels([f.replace('_', ' ').title() for f in sample_features])
            ax.set_ylim(0, 1)
            ax.set_title('Cluster Comparison - All Segments', size=16, weight='bold', pad=30)
            ax.grid(True, alpha=0.3)
            ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
            
            plt.tight_layout()
            plt.show()
        
        # Individual cluster charts with automatic grid layout
        n_clusters = len(normalized_means)
        
        # Calculate optimal grid dimensions
        if n_clusters <= 3:
            n_rows, n_cols = 1, n_clusters
        elif n_clusters <= 6:
            n_rows, n_cols = 2, (n_clusters + 1) // 2
        elif n_clusters <= 9:
            n_rows, n_cols = 3, (n_clusters + 2) // 3
        else:
            n_rows = int(np.ceil(np.sqrt(n_clusters)))
            n_cols = int(np.ceil(n_clusters / n_rows))
        
        # Adjust figure size based on grid
        fig_width = min(5 * n_cols, 20)
        fig_height = min(5 * n_rows, 15)
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), 
                               subplot_kw=dict(projection='polar'))
        
        # Handle different cases for axes array structure
        if n_clusters == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = axes if n_cols > 1 else [axes]
        else:
            axes = axes.flatten()
        
        angles = np.linspace(0, 2*np.pi, len(sample_features), endpoint=False).tolist()
        angles += angles[:1]  # Complete the circle
        
        colors = plt.cm.Set3(np.linspace(0, 1, n_clusters))
        
        for idx, (cluster_id, cluster_data) in enumerate(normalized_means.iterrows()):
            values = cluster_data.tolist()
            values += values[:1]  # Complete the circle
            
            ax = axes[idx]
            ax.plot(angles, values, 'o-', linewidth=3, color=colors[idx])
            ax.fill(angles, values, alpha=0.25, color=colors[idx])
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels([f.replace('_', ' ').title()[:15] for f in sample_features], fontsize=10)
            ax.set_ylim(0, 1)
            
            # Add cluster size info
            cluster_size = len(df_filtered[df_filtered['Cluster'] == cluster_id])
            cluster_pct = cluster_size / len(df_filtered) * 100
            ax.set_title(f'Cluster {cluster_id}\n({cluster_size} customers, {cluster_pct:.1f}%)', 
                        size=12, weight='bold', pad=20)
            ax.grid(True, alpha=0.3)
        
        # Hide unused subplots if any
        total_subplots = n_rows * n_cols
        if n_clusters < total_subplots:
            for i in range(n_clusters, total_subplots):
                axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    def create_cluster_stability_analysis(self, methods=['HDBSCAN', 'K-Means'], n_iterations=20):
        """
        Analyze cluster stability across multiple runs
        """
        print("Performing cluster stability analysis...")
        
        stability_results = {}
        
        for method in methods:
            if method not in self.cluster_results:
                continue
                
            print(f"Analyzing stability for {method}...")
            
            # Get the original clustering
            original_labels = self.cluster_results[method]['labels']
            n_clusters = self.cluster_results[method]['n_clusters']
            
            # Perform multiple clusterings with bootstrap sampling
            stability_scores = []
            
            for i in range(n_iterations):
                # Bootstrap sample
                n_samples = len(self.processed_data)
                indices = np.random.choice(n_samples, size=int(0.8 * n_samples), replace=True)
                bootstrap_data = self.processed_data[indices]
                
                # Apply clustering
                try:
                    if method == 'HDBSCAN':
                        clusterer = hdbscan.HDBSCAN(
                            min_cluster_size=max(5, len(bootstrap_data) // 50),
                            min_samples=3
                        )
                        bootstrap_labels = clusterer.fit_predict(bootstrap_data)
                    elif method == 'K-Means':
                        clusterer = KMeans(n_clusters=n_clusters, random_state=i, n_init=10)
                        bootstrap_labels = clusterer.fit_predict(bootstrap_data)
                    elif method == 'Gaussian Mixture':
                        clusterer = GaussianMixture(n_components=n_clusters, random_state=i)
                        clusterer.fit(bootstrap_data)
                        bootstrap_labels = clusterer.predict(bootstrap_data)
                    else:
                        continue
                    
                    # Calculate adjusted rand index with original clustering (subset)
                    original_subset = original_labels[indices]
                    if len(np.unique(bootstrap_labels)) > 1 and len(np.unique(original_subset)) > 1:
                        stability_score = adjusted_rand_score(original_subset, bootstrap_labels)
                        stability_scores.append(stability_score)
                
                except Exception as e:
                    continue
            
            if stability_scores:
                stability_results[method] = {
                    'mean_stability': np.mean(stability_scores),
                    'std_stability': np.std(stability_scores),
                    'scores': stability_scores
                }
        
        # Plot stability results
        if stability_results:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Bar plot of mean stability
            methods = list(stability_results.keys())
            means = [stability_results[m]['mean_stability'] for m in methods]
            stds = [stability_results[m]['std_stability'] for m in methods]
            
            ax1.bar(methods, means, yerr=stds, capsize=5, alpha=0.7)
            ax1.set_title('Cluster Stability Comparison')
            ax1.set_ylabel('Mean Adjusted Rand Index')
            ax1.set_ylim(0, 1)
            ax1.grid(True, alpha=0.3)
            
            # Distribution plot
            for i, method in enumerate(methods):
                scores = stability_results[method]['scores']
                ax2.hist(scores, alpha=0.6, label=method, bins=10)
            
            ax2.set_title('Stability Score Distributions')
            ax2.set_xlabel('Adjusted Rand Index')
            ax2.set_ylabel('Frequency')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Print results
            print(f"\n{'='*50}")
            print("STABILITY ANALYSIS RESULTS")
            print(f"{'='*50}")
            
            for method, results in stability_results.items():
                mean_stab = results['mean_stability']
                std_stab = results['std_stability']
                print(f"{method}: {mean_stab:.3f} ± {std_stab:.3f}")
                
                if mean_stab >= 0.7:
                    print(f"  ✓ Highly stable clustering")
                elif mean_stab >= 0.5:
                    print(f"  ✓ Moderately stable clustering")
                else:
                    print(f"  ⚠ Low stability - consider parameter tuning")
        
        return stability_results    
        
    def evaluate_clusters(self):
        """
        Enhanced cluster evaluation with business validation
        """
        print("\n" + "="*70)
        print("COMPREHENSIVE CLUSTERING EVALUATION")
        print("="*70)
        
        evaluation_results = []
        
        for method, result in self.cluster_results.items():
            labels = result['labels']
            n_clusters = result['n_clusters']
            
            # Basic metrics
            sil_score = result.get('silhouette_score', -1)
            ch_score = result.get('calinski_harabasz_score', 0)
            db_score = result.get('davies_bouldin_score', float('inf'))
            stability = result.get('stability_score', 0)
            
            # Business validation metrics
            cluster_sizes = np.bincount(labels[labels >= 0])  # Exclude noise points
            min_cluster_size_pct = (np.min(cluster_sizes) / len(labels)) * 100 if len(cluster_sizes) > 0 else 0
            
            # Actionability score (based on feature separation)
            actionability_score = self.calculate_actionability_score(labels)
            
            # Overall score (weighted combination)
            if sil_score > 0:
                overall_score = (
                    0.3 * sil_score +
                    0.2 * min(ch_score / 1000, 1) +  # Normalized CH score
                    0.2 * max(0, 1 - db_score / 5) +  # Inverted DB score
                    0.15 * stability +
                    0.15 * actionability_score
                )
            else:
                overall_score = 0
            
            evaluation_results.append({
                'Method': method,
                'Clusters': n_clusters,
                'Silhouette': f"{sil_score:.3f}" if sil_score > 0 else "N/A",
                'CH Score': f"{ch_score:.1f}" if ch_score > 0 else "N/A",
                'DB Score': f"{db_score:.3f}" if db_score != float('inf') else "N/A",
                'Stability': f"{stability:.3f}",
                'Min Size %': f"{min_cluster_size_pct:.1f}%",
                'Actionability': f"{actionability_score:.3f}",
                'Overall Score': f"{overall_score:.3f}",
                'Noise Points': result.get('noise_points', 0)
            })
        
        eval_df = pd.DataFrame(evaluation_results)
        print(eval_df.to_string(index=False))
        
        # Business validation checks
        print(f"\n{'='*50}")
        print("BUSINESS VALIDATION CHECKS")
        print(f"{'='*50}")
        
        for method, result in self.cluster_results.items():
            labels = result['labels']
            cluster_sizes = np.bincount(labels[labels >= 0])
            
            print(f"\n{method}:")
            print(f"  ✓ Number of clusters: {result['n_clusters']}")
            
            # Size validation
            min_size_pct = (np.min(cluster_sizes) / len(labels)) * 100 if len(cluster_sizes) > 0 else 0
            if min_size_pct >= 5:
                print(f"  ✓ All clusters >5% of population (min: {min_size_pct:.1f}%)")
            else:
                print(f"  ✗ Some clusters <5% of population (min: {min_size_pct:.1f}%)")
            
            # Stability validation
            stability = result.get('stability_score', 0)
            if stability >= 0.3:
                print(f"  ✓ Good stability (score: {stability:.3f})")
            else:
                print(f"  ✗ Low stability (score: {stability:.3f})")
        
        # Recommend best method
        valid_methods = {k: v for k, v in evaluation_results if v['Overall Score'] != '0.000'}
        
        if valid_methods:
            best_method_row = max(evaluation_results, key=lambda x: float(x['Overall Score']))
            best_method = best_method_row['Method']
            print(f"\n🏆 RECOMMENDED METHOD: {best_method}")
            print(f"   Overall Score: {best_method_row['Overall Score']}")
            print(f"   Clusters:        print(f"Suggested optimal number of clusters: {optimal_k}")
        
        # Also calculate Gap Statistic
        gap_optimal_k, _ = self.gap_statistic(max_clusters=max_clusters)
        
        print(f"Gap Statistic suggests: {gap_optimal_k}")
        print(f"Final recommendation: {optimal_k} (based on silhouette score)")
        
        return optimal_k        self.preprocessing_info = {}
        
    def preprocess_data(self, categorical_cols=None, numerical_cols=None, use_pca=True, pca_variance_threshold=0.95):
        """
        Enhanced preprocessing with composite features and advanced encoding
        """
        print("Starting enhanced data preprocessing...")
        
        # Create composite features first
        self.create_composite_features()
        
        # Auto-detect column types if not specified
        if categorical_cols is None:
            categorical_cols = self.df.select_dtypes(include=['object', 'category']).columns.tolist()
        if numerical_cols is None:
            numerical_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Identify ordinal features
        ordinal_features = {}
        if 'spending_score' in categorical_cols:
            ordinal_features['spending_score'] = ['Low', 'Average', 'High']
        if 'education' in categorical_cols:
            ordinal_features['education'] = ['High School', 'Bachelor', 'Master', 'PhD']
        if 'income_bracket' in categorical_cols:
            ordinal_features['income_bracket'] = ['Low', 'Medium', 'High']
        
        # Separate ordinal from regular categorical
        ordinal_cols = list(ordinal_features.keys())
        regular_categorical = [col for col in categorical_cols if col not in ordinal_cols]
        
        self.preprocessing_info = {
            'categorical_cols': regular_categorical,
            'numerical_cols': numerical_cols,
            'ordinal_cols': ordinal_cols,
            'ordinal_mappings': ordinal_features
        }
        
        # Create preprocessing transformers
        transformers = []
        
        # Numerical features - use RobustScaler for outlier resistance
        if numerical_cols:
            transformers.append(('num', RobustScaler(), numerical_cols))
        
        # Ordinal features
        for col, categories in ordinal_features.items():
            if col in self.df.columns:
                transformers.append((f'ord_{col}', OrdinalEncoder(categories=[categories]), [col]))
        
        # Regular categorical features - OneHot encoding
        if regular_categorical:
            # Filter out high cardinality features (>10 unique values)
            low_card_categorical = []
            for col in regular_categorical:
                if col in self.df.columns and self.df[col].nunique() <= 10:
                    low_card_categorical.append(col)
            
            if low_card_categorical:
                transformers.append(('cat', OneHotEncoder(drop='first', sparse_output=False), low_card_categorical))
        
        # Apply preprocessing
        if transformers:
            preprocessor = ColumnTransformer(transformers=transformers, remainder='drop')
            processed_data = preprocessor.fit_transform(self.df)
            self.preprocessor = preprocessor
            
            # Get feature names
            feature_names = []
            for name, transformer, cols in transformers:
                if name == 'num':
                    feature_names.extend(cols)
                elif name.startswith('ord_'):
                    feature_names.extend(cols)
                elif name == 'cat':
                    if hasattr(transformer, 'get_feature_names_out'):
                        feature_names.extend(transformer.get_feature_names_out(cols))
                    else:
                        feature_names.extend(cols)
            
            self.feature_names = feature_names
        else:
            processed_data = self.df.select_dtypes(include=[np.number]).values
            self.feature_names = self.df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Apply PCA if requested
        if use_pca and processed_data.shape[1] > 5:
            print("Applying PCA for dimensionality reduction...")
            self.pca = PCA()
            pca_data = self.pca.fit_transform(processed_data)
            
            # Determine number of components based on variance threshold
            cumsum_var = np.cumsum(self.pca.explained_variance_ratio_)
            n_components = np.argmax(cumsum_var >= pca_variance_threshold) + 1
            
            print(f"PCA: Using {n_components} components explaining {cumsum_var[n_components-1]:.1%} of variance")
            
            # Refit with optimal components
            self.pca = PCA(n_components=n_components)
            self.processed_data = self.pca.fit_transform(processed_data)
            self.use_pca = True
            
            # Store both versions
            self.processed_data_original = processed_data
            self.pca_feature_names = [f'PC{i+1}' for i in range(n_components)]
        else:
            self.processed_data = processed_data
            self.use_pca = False
            self.processed_data_original = processed_data
        
        print(f"Final processed data shape: {self.processed_data.shape}")
        return self.processed_data


SyntaxError: unterminated string literal (detected at line 1430) (3453487636.py, line 1430)

In [8]:

# Enhanced usage function with comprehensive analysis
def run_enhanced_clustering_analysis(df, categorical_cols=None, numerical_cols=None, 
                                    use_pca=True, target_clusters=None):
    """
    Main function to run enhanced clustering analysis with all improvements
    """
    print("🚀 Starting Enhanced Demographic Clustering Analysis")
    print("="*70)
    
    # Initialize clusterer
    clusterer = DemographicClusterer(df=df)
    
    # Enhanced preprocessing with composite features
    clusterer.preprocess_data(categorical_cols=categorical_cols, 
                            numerical_cols=numerical_cols, 
                            use_pca=use_pca)
    
    # Find optimal number of clusters using multiple methods
    print(f"\n📊 Finding optimal number of clusters...")
    optimal_k = clusterer.find_optimal_clusters(max_clusters=8)
    
    if target_clusters is not None:
        optimal_k = target_clusters
        print(f"Using specified target clusters: {optimal_k}")
    
    # Apply enhanced clustering methods (HDBSCAN priority)
    print(f"\n🔍 Applying clustering algorithms...")
    results = clusterer.apply_clustering_methods(n_clusters=optimal_k)
    
    # Comprehensive evaluation
    print(f"\n📈 Evaluating clustering quality...")
    evaluation = clusterer.evaluate_clusters()
    
    # Find best method based on overall score
    best_method = 'HDBSCAN'  # Default to HDBSCAN
    if not evaluation.empty:
        best_row = evaluation.loc[evaluation['Overall Score'].astype(float).idxmax()]
        best_method = best_row['Method']
    
    print(f"\n🎯 Primary analysis using: {best_method}")
    
    # Advanced visualizations and analysis
    methods_to_analyze = [best_method]
    if best_method != 'HDBSCAN' and 'HDBSCAN' in results:
        methods_to_analyze.append('HDBSCAN')
    if best_method != 'K-Means' and 'K-Means' in results:
        methods_to_analyze.append('K-Means')
    
    for method in methods_to_analyze:
        if method in results:
            print(f"\n" + "="*50)
            print(f"DETAILED ANALYSIS: {method}")
            print("="*50)
            
            # 1. Create cluster profiles
            print(f"\n📋 Creating cluster profiles...")
            df_with_clusters = clusterer.create_cluster_profiles(method)
            
            # 2. Generate segment names
            print(f"\n🏷️ Generating segment names...")
            segment_names = clusterer.generate_segment_names(method)
            
            # 3. Advanced t-SNE visualization
            print(f"\n📊 Creating advanced t-SNE visualizations...")
            clusterer.create_advanced_tsne_visualization(method, perplexity_values=[30, 50])
            
            # 4. Enhanced radial charts
            print(f"\n🎭 Creating enhanced radial charts...")
            clusterer.create_enhanced_radial_charts(method, show_comparison=True)
            
            # 5. Feature contribution analysis
            print(f"\n🔍 Analyzing feature contributions...")
            clusterer.create_feature_contribution_plot(method)
            
            # 6. Business dashboard
            print(f"\n📈 Creating business dashboard...")
            clusterer.create_business_dashboard(method)
            
            # 7. Comparison heatmap
            print(f"\n🌡️ Creating comparison heatmap...")
            clusterer.create_comparison_heatmap(method)
            
            print(f"\n" + "-"*50)
    
    # Stability analysis across methods
    print(f"\n🔬 Performing stability analysis...")
    stability_results = clusterer.create_cluster_stability_analysis(methods=list(results.keys())[:3])
    
    # Final recommendations
    print(f"\n" + "="*70)
    print("🎯 FINAL RECOMMENDATIONS")
    print("="*70)
    
    print(f"Best Method: {best_method}")
    if best_method in results:
        best_result = results[best_method]
        print(f"Number of Clusters: {best_result['n_clusters']}")
        if 'silhouette_score' in best_result:
            print(f"Silhouette Score: {best_result['silhouette_score']:.3f}")
        if 'noise_points' in best_result:
            print(f"Noise Points: {best_result['noise_points']}")
    
    print(f"\n💡 Business Insights:")
    if best_method in results:
        labels = results[best_method]['labels']
        valid_clusters = len(set(labels[labels >= 0]))
        
        if valid_clusters >= 4 and valid_clusters <= 6:
            print("  ✓ Optimal number of segments for business strategy")
        elif valid_clusters < 4:
            print("  ⚠ Consider increasing clusters for more granular insights")
        else:
            print("  ⚠ Consider reducing clusters for simpler strategy")
        
        # Check if we have the target segments
        df_with_clusters = df.copy()
        df_with_clusters['Cluster'] = labels
        df_filtered = df_with_clusters[df_with_clusters['Cluster'] >= 0]
        
        if len(df_filtered) > 0:
            min_cluster_size = df_filtered['Cluster'].value_counts().min()
            min_pct = (min_cluster_size / len(df_filtered)) * 100
            
            if min_pct >= 5:
                print("  ✓ All segments are actionable (>5% of population)")
            else:
                print("  ⚠ Some segments may be too small for targeted strategies")
    
    print(f"\n🎯 Next Steps:")
    print("  1. Review segment characteristics and create detailed personas")
    print("  2. Develop targeted marketing strategies for each segment")
    print("  3. Validate segments with business stakeholders")
    print("  4. Monitor segment stability over time")
    print("  5. Consider A/B testing different approaches per segment")
    
    return clusterer, results, best_method


In [ ]:

# Run analysis on enhanced sample data
if __name__ == "__main__":
    print("Creating enhanced sample demographic data...")
    sample_df = create_enhanced_sample_demographic_data(1000)
    print("Enhanced sample data created successfully!")
    print(f"Data shape: {sample_df.shape}")
    print("\nSample data preview:")
    print(sample_df.head())
    print("\nData info:")
    print(sample_df.info())
    
    # Run enhanced clustering analysis
    print("\n" + "="*70)
    print("STARTING ENHANCED CLUSTERING ANALYSIS")
    print("="*70)
    
    clusterer, results, best_method = run_enhanced_clustering_analysis(
        sample_df, 
        use_pca=True, 
        target_clusters=5  # Target the 5 business segments
    )

In [9]:
import pandas as pd
datapath = "/Users/whysocurious/Documents/MLDSAIProjects/cust-seg-case-study/data"
dfcust = pd.read_csv(datapath+'/customer_data_imputed.csv').set_index('Customer ID')
#(datapath+'/Customer Segmentation.csv')
print (f"Data shape: {dfcust.shape}")
print (f"Data columns: {dfcust.columns.tolist()}")
print (f"Data types:\n{dfcust.dtypes}")
print (f"Missing values:\n{dfcust.isnull().sum()}")
# dfcust.head()

Data shape: (10695, 8)
Data columns: ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession', 'Work_Experience', 'Spending_Score', 'Family_Size']
Data types:
Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
dtype: object
Missing values:
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
dtype: int64


In [10]:
clusterer, results, best_method = run_enhanced_clustering_analysis(
        dfcust, 
        use_pca=True, 
        target_clusters=5  # Target the 5 business segments
    )

🚀 Starting Enhanced Demographic Clustering Analysis


AttributeError: 'DemographicClusterer' object has no attribute 'preprocess_data'